## EIA International

This notebook : 
- downloads the bulk file on EIA open data portal
- opens the txt file containing JSONs and convert it into a DataFrame using pandas
- clean the data and structure it the way we want
- then, store it into a csv file named "../../data/_processed/eia_international_processed.csv"

In [11]:
import pandas as pd
import urllib.request
import json
import zipfile
import time
import os

In [12]:
# download du fichier international .zip sur eia (https://www.eia.gov/opendata/)

# url of the international file on eia
url = "https://www.eia.gov/opendata/bulk/INTL.zip"


#/Users/alexandrebernard/Documents/Perso/Data/D4G/shiftdataportal_data
#defining file name and destination_raw
destination_raw = "../../data/_raw/eia/"
destination_processed = "../../data/_processed/" 
file_name = "eia_international_bulk.zip"

### Bulk Download of the txt file containing all data of International API route

In [13]:
#download the file from url
urllib.request.urlretrieve(url, destination_raw + file_name)
print("download completed")

download completed


### unzip the file

In [14]:
# unzip the file and delete zip

#ouvrir le fichier en mode lecture
with zipfile.ZipFile(destination_raw + file_name, 'r') as zip_ref:
    #Extract file in same repertory
    zip_ref.extractall(destination_raw)
print("file unzipped")

file unzipped


### Extraction of the JSONs from txt file into a dataframe

In [15]:

start_time = time.time()

json_list = []

print("---open txt file---")
#open the file in reading mode
with open(destination_raw + "INTL.txt", 'r') as file_txt:

    count = 0                        # TO DELETE
    # iteration on each json in the txt file
    for row in file_txt:
        #delete blank and new line caracter
        row = row.strip()
        #load the txt of one row into a json. use of "json.loads()" as we are loading a string into a json file corresponding to the row
        data_json = json.loads(row)
        #insert in dataframe
        #df_temp = pd.DataFrame(data_json)
        #add to the list of jsons
        json_list.append(data_json)
        
        #management of execution time
        count += 1                   # TO DELETE
        if count % 5000 == 0:
            print("---iteration on %s row completed---" %count)
        #if stop == 20:              # TO DELETE
        #    break                   # TO DELETE

df = pd.DataFrame(json_list)

iteration_time = time.time()
print("--- iteration done in %s seconds ---" %(iteration_time - start_time))


---open txt file---


---iteration on 5000 row completed---
---iteration on 10000 row completed---
---iteration on 15000 row completed---
---iteration on 20000 row completed---
---iteration on 25000 row completed---
---iteration on 30000 row completed---
---iteration on 35000 row completed---
---iteration on 40000 row completed---
---iteration on 45000 row completed---
---iteration on 50000 row completed---
---iteration on 55000 row completed---
---iteration on 60000 row completed---
---iteration on 65000 row completed---
---iteration on 70000 row completed---
---iteration on 75000 row completed---
---iteration on 80000 row completed---
---iteration on 85000 row completed---
---iteration on 90000 row completed---
---iteration on 95000 row completed---
---iteration on 100000 row completed---
--- iteration done in 19.729034900665283 seconds ---


### creating the different columns

In [18]:
#deletion of NaN values
df_structured = df.dropna(subset=["data"])

#we only keep annual data
df_structured = df_structured[df_structured["f"]=="A"]

#management of the columns "data" that is a list of lists
df_structured = df_structured.explode("data")

# transformation of the "data" columns (containing a list [year, value] into 2 columns for date and value
df_structured[["date", "value"]] = pd.DataFrame(df_structured["data"].tolist(), index=df_structured.index)
df_structured = df_structured.drop("data", axis=1)

#split of concatenated columns for the "series_id" column
df_structured[["file_name", "product_id", "activity_id", "country_region_id", "unit_id", "frequency_id"]] = df_structured["series_id"].str.split('[.-]', expand=True)
# "name" column not working because there can be 2 until 7 ","

#we do not keep regional data.
df_structured = df_structured[df_structured["country_region_id"].str.len()!=4]
#df_structured.shape (3 952 091, 23) puis (3 364 396) après suppr des regions
# Warning : Following countrie have 4 caracter for there id : DEUW allemagne ouest, DEUO GM offshore used for load only, 
# GBRO UK offshore used for load only, NLDA Netherlands antilles, NLDO NL offshore used for load only
# USIQ US pacific islands, USOH us territories

df_structured = df_structured.drop_duplicates(["name", "country_region_id", "activity_id", "date", "unit_id"])
#=> shape of the df before droping duplicates : (3364396, 11). After deleting duplicates : (3345590, 11)

#reset of indexes
df_structured = df_structured.reset_index(drop=True)

### columns renaming and selection, value column cleaning, country in lower case

In [35]:
#selection and reordering of the columns
df_eia_cleaned = df_structured[["source", "file_name", "name", "country_region_id", "product_id", "activity_id", "date", "unit_id", "units", "value", "last_updated"]]

#colums renaming
df_eia_cleaned.rename(columns={
    'units': 'unit_name',
    'name': 'product_region_freq_name',
    'date': 'year',
    'country_region_id': 'country_id'
}, inplace=True)


#clean value column
values_to_replace = ["--", "ie", "w", "NA"]
df_eia_cleaned.loc[:,"value"] = df_eia_cleaned["value"].replace(values_to_replace, 0)

#conversion of value into float data type
df_eia_cleaned.loc[:,"value"] = df_eia_cleaned["value"].astype(float)

#conversion of value into float data type
df_eia_cleaned.loc[:,"country_id"] = df_eia_cleaned["country_id"].str.lower()

#conversion of product_id into int data type
df_eia_cleaned.loc[:,("product_id", "activity_id")] = df_eia_cleaned[["product_id", "activity_id"]].astype(int)

/var/folders/pd/8mtfl9m11sbf2r8nhg8s0jc80000gp/T/ipykernel_19070/1090340945.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_eia_cleaned.rename(columns={
/var/folders/pd/8mtfl9m11sbf2r8nhg8s0jc80000gp/T/ipykernel_19070/1090340945.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_eia_cleaned.loc[:,"value"] = df_eia_cleaned["value"].replace(values_to_replace, 0)
/var/folders/pd/8mtfl9m11sbf2r8nhg8s0jc80000gp/T/ipykernel_19070/1090340945.py:15: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace inste

## Add external data (product name, activity name)

In [41]:
# load of the product names
#data (product_name, product_id) come from a copy paste on the API browser
#this data processed in this notebook : notebooks/raw/ab_eia_raw_product_name_id.ipynb
df_product = pd.read_csv("../../data/_processed/eia_product_name_id.csv")

# load of activity names
# As it's complicated to extract activity text, let's build the data from data in the API exploration browser
df_activity = pd.DataFrame({"activity_id" : [1, 2, 3, 4, 5, 6, 7, 8, 9, 12, 13, 23, 33, 34],
                            "activity_name" : ["Production", "Consumption", "Imports", "Exports", "Stocks, OECD", "Reserves", "Capacity", "Emissions", "Distribution losses", "Generation", "Bunker", "Net Imports", "Population", "GDP"]})

# load of the countries
df_country = pd.read_csv("../../data/countries.csv")

# merge of the product names
df_eia = df_eia_cleaned.merge(df_product, left_on="product_id", right_on="product_id", how="left")

# merge of the activity names
df_eia = df_eia.merge(df_activity, left_on="activity_id", right_on="activity_id", how="left")

# merge of the countries
df_eia = df_eia.merge(df_country[["alpha3", "name"]], left_on="country_id", right_on="alpha3", how="left")
#country name colums renaming and drop of alpha3 column
df_eia.rename(columns={'name': 'country_name'}, inplace=True)
df_eia.drop("alpha3", axis=1)

#columns reordering
df_eia = df_eia[['source', 'file_name', 'product_region_freq_name', 'country_id',
                 'country_name','product_id', 'product_name', 'activity_id', 'activity_name'
                 , 'year', 'unit_id', 'unit_name', 'value','last_updated']]

### store cleaned eia data in csv file + delete zip and unzipped file

In [43]:
#index on false to not send it into the csv file
df_eia.to_csv(destination_processed + "eia_international_processed.csv", index=False)

os.remove(destination_raw + "eia_international_bulk.zip")
os.remove(destination_raw + "INTL.txt")